# 1. Install neccesarry libraries

In [14]:
 # This is a library for image reshaping
!pip install Pillow 

#Import neccesary libraries
import numpy as np 
import pandas as pd 

# 2. Read the csv files

We will have 2 different csv files. 

First one *'GroundTruth.csv '* contains images_id with their labes.

And the other one *'HAM10000_metadata.csv'* is contains a informations regading the image itself.



In [68]:
#Read the csv files into a dataframe

df = pd.read_csv('GroundTruth.csv')

df2 = pd.read_csv('HAM10000_metadata.csv')

df.head(5)

#!ls "./images/images" #Before preprocessing checked directory 

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   image   10015 non-null  object 
 1   MEL     10015 non-null  float64
 2   NV      10015 non-null  float64
 3   BCC     10015 non-null  float64
 4   AKIEC   10015 non-null  float64
 5   BKL     10015 non-null  float64
 6   DF      10015 non-null  float64
 7   VASC    10015 non-null  float64
dtypes: float64(7), object(1)
memory usage: 626.1+ KB


In [69]:
df2.head(5)

df2.info()

#As we can see in df2 the column with image_id is different than the one in df.
#We need to change the column name in df2 to match the one in df

df2 = df2.rename(columns={'image_id': 'image'},inplace=False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lesion_id     10015 non-null  object 
 1   image_id      10015 non-null  object 
 2   dx            10015 non-null  object 
 3   dx_type       10015 non-null  object 
 4   age           9958 non-null   float64
 5   sex           10015 non-null  object 
 6   localization  10015 non-null  object 
dtypes: float64(1), object(6)
memory usage: 547.8+ KB


In [70]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lesion_id     10015 non-null  object 
 1   image         10015 non-null  object 
 2   dx            10015 non-null  object 
 3   dx_type       10015 non-null  object 
 4   age           9958 non-null   float64
 5   sex           10015 non-null  object 
 6   localization  10015 non-null  object 
dtypes: float64(1), object(6)
memory usage: 547.8+ KB


In order to make a analysis based on the images we can concatenate both dataframes to one 

In [72]:
#Merge the two dataframes on the image_id column

df3 = pd.merge(df, df2, on='image')

df3.head(5)

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC,lesion_id,dx,dx_type,age,sex,localization
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0,HAM_0000550,nv,follow_up,45.0,male,trunk
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0,HAM_0003577,nv,follow_up,50.0,male,lower extremity
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0,HAM_0001477,nv,follow_up,55.0,female,trunk
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0,HAM_0000484,nv,follow_up,40.0,male,trunk
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0,HAM_0003350,mel,histo,60.0,male,chest


In [31]:
#Show the number of columns we have to proccess
print(len(df))

#Show the columns we have to proccess
print(df.columns)

#drop the image column from the dataframe so that create a label array

labels = df.drop(columns=['image'])

10015
Index(['image', 'MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'], dtype='object')


In [33]:
#From above we can see that our dataframe is not contain .jpg extension so we have to add it to the dataframe
#This will help us to read the images from the directory

df['image'] = df['image'].apply(lambda x: x + '.jpg')

print(df.head())

              image  MEL   NV  BCC  AKIEC  BKL   DF  VASC
0  ISIC_0024306.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
1  ISIC_0024307.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
2  ISIC_0024308.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
3  ISIC_0024309.jpg  0.0  1.0  0.0    0.0  0.0  0.0   0.0
4  ISIC_0024310.jpg  1.0  0.0  0.0    0.0  0.0  0.0   0.0


# 3. Preprocessing the images

Resizing images will be helpful for us to work on models.

This images are formed as RGB which is Red,Green and Blue Channels.

When re-size the images as (128,128) it will create a a column as 128x128x3 = 49152 as showed in resized_images[0]

In [95]:
import os
from IPython.display import Image, display

from PIL import Image

image_directory = './images/images/' 

#We need to end of the file as jnp because in the directory there is .txt file
image_files = [os.path.join(image_directory, file) for file in os.listdir(image_directory) if file.endswith('.jpg')]

#Sort the images based on their indexis
image_files.sort()

count = 0

#Resize the images to 128x128
resized_images = []
for image_file in image_files:
    
    with Image.open(image_file) as img:
    
        img = img.resize((128,128))
        #Make sure that the image is RGB and convert it to numpy array
        img_array = np.array(img)

        #Normalize the image
        img = img_array/ 255.0


        #print(img.size)
        resized_images.append(img_array)


#This shows the shape of my images and the min and max values of the pixels
#This will help us to see if there is any problem with the images
img_array.shape, img_array.min(), img_array.max()

((128, 128, 3), 15, 239)

In [96]:
#This convertion is neccesarry since we are feeding the images to the neural network as numpy array 
# numpy make it as 4 dimensional array, before proccesing we had 3 dimensional array
X = np.array(resized_images)

X.shape

(10015, 128, 128, 3)

## 3.2 Preprocessing the One-hot encoding Y labels

In [104]:
#One hot encoding for the labels expect the image column from first dataframe
y = pd.get_dummies(df.drop(columns=['image']))

#Create a one hot encoded array for  Numpy Array    
y = np.array(y)



AttributeError: 'numpy.ndarray' object has no attribute 'value'